Examples of Loading Term Structure (futures and spot vix indexes)

In [ ]:
import os

import numpy as np
import pandas as pd

import plotly.express as px

In [ ]:
base_data_path = '/Users/aptperson/source/trading/vix_utils/futures/download/monthly'
fns = os.listdir(base_data_path)
fns = [f for f in fns if f.endswith('.csv')]
# fns = [f for f in fns if '_VXM_' not in f]
fns = [f for f in fns if '_VX_' not in f]
df = pd.concat([pd.read_csv(os.path.join(base_data_path, f)) for f in fns])
df.index = pd.to_datetime(df['Trade Date'])
df.index.name = 'date'
df['ret'] = np.log(df.Close)
df['ret'] = df.groupby('Futures').ret.diff().shift(-1)
df

In [ ]:
px.line(df, y='Close', color='Futures')

In [ ]:
px.line(df, y='Total Volume', color='Futures')

In [ ]:
#### continuous futures series

# assume that the end of file is the last trading day
# N days before, roll
# backwardation/contango

# volume roll

In [ ]:
# last day for each contract
N = 3
expiry_dates = df.reset_index().groupby('Futures').date.max().to_frame('expiry').reset_index().sort_values(['expiry', 'Futures'])
roll_dates = expiry_dates.copy()
roll_dates['roll'] = roll_dates['expiry'] - pd.Timedelta(days=N)
roll_dates['roll_to'] = roll_dates.roll.shift(1)
roll_dates.drop_duplicates(subset=['expiry', 'roll', 'roll_to'], inplace=True)
# roll_dates.dropna(inplace=True)
roll_dates['next_future'] = roll_dates.Futures.shift(-1)
roll_dates['n2_future'] = roll_dates.Futures.shift(-2)
roll_dates = roll_dates.loc[roll_dates.roll > roll_dates.roll_to]
roll_dates.tail(10)


In [ ]:
continuous_df = pd.DataFrame(index=df.reset_index().date.unique()).sort_index()
continuous_df.index.name = 'date'
continuous_df = pd.merge_asof(continuous_df, left_index=True, right=roll_dates, right_on='roll_to', direction='backward')#.tail(50)
continuous_df = pd.merge(continuous_df.reset_index(), on=['Futures', 'date'], right = df.reset_index()[['date', 'Futures', 'ret', 'Close']], how='left')

continuous_df = pd.merge(continuous_df.reset_index(), left_on=['next_future', 'date'], right = df.reset_index()[['date', 'Futures', 'Close']], right_on=['Futures', 'date'], suffixes=['', '_next'], how='left')
continuous_df = pd.merge(continuous_df, left_on=['n2_future', 'date'], right = df.reset_index()[['date', 'Futures', 'Close']], right_on=['Futures', 'date'], suffixes=['', '_n2'], how='left')

continuous_df['cpnl'] = -continuous_df.ret.cumsum()
continuous_df.set_index('date', inplace=True)
continuous_df.dropna(subset=['ret'], inplace=True)

sr = -continuous_df.ret.mean() / continuous_df.ret.std() * np.sqrt(252)
print(f'SR: {sr:.2f}')

years = continuous_df.index.values[-1] - continuous_df.index.values[0]
years = int(years) / (365 * 24 * 60 * 60) / 1e9
cagr = (continuous_df.cpnl.values[-1]) ** (1/years) - 1

print(f'CAGR: {cagr*100:.1f}% ({years:.1f}years) ({continuous_df.cpnl.values[-1]:.1f}) Wealth multiple')

px.line(continuous_df, y='cpnl')

In [ ]:
continuous_df['term_structure'] = 'contango'
continuous_df.loc[continuous_df.Close > continuous_df.Close_next, 'term_structure'] = 'backwardation'

continuous_df['term_structure_n2'] = 'contango'
continuous_df.loc[continuous_df.Close_next > continuous_df.Close_n2, 'term_structure_n2'] = 'backwardation'

continuous_df.groupby(['term_structure', 'term_structure_n2']).ret.describe()

In [ ]:
px.histogram(continuous_df, x='ret', color='term_structure_n2')

In [ ]:
continuous_df = pd.DataFrame(index=df.reset_index().date.unique()).sort_index()
continuous_df.index.name = 'date'
continuous_df = pd.merge_asof(continuous_df, left_index=True, right=roll_dates, right_on='roll_to', direction='backward')#.tail(50)
continuous_df = pd.merge(continuous_df.reset_index(), on=['Futures', 'date'], right = df.reset_index()[['date', 'Futures', 'ret', 'Close']], how='left')
continuous_df = pd.merge(continuous_df.reset_index(), left_on=['next_future', 'date'], right = df.reset_index()[['date', 'Futures', 'Close']], right_on=['Futures', 'date'], suffixes=['', '_next'], how='left')
continuous_df = pd.merge(continuous_df, left_on=['n2_future', 'date'], right = df.reset_index()[['date', 'Futures', 'Close']], right_on=['Futures', 'date'], suffixes=['', '_n2'], how='left')


continuous_df['term_structure'] = 'contango'
continuous_df.loc[continuous_df.Close > continuous_df.Close_next, 'term_structure'] = 'backwardation'

continuous_df['term_structure_n2'] = 'contango'
continuous_df.loc[continuous_df.Close_next > continuous_df.Close_n2, 'term_structure_n2'] = 'backwardation'

# continuous_df.loc[continuous_df.term_structure == 'backwardation', 'ret'] = 0
mask = ((continuous_df.term_structure == 'contango') & (continuous_df.term_structure_n2 == 'contango')) | ((continuous_df.term_structure == 'backwardation') & (continuous_df.term_structure_n2 == 'backwardation')) 
continuous_df.loc[~mask, 'ret'] = 0

continuous_df['cpnl'] = -continuous_df.ret.cumsum()
continuous_df.set_index('date', inplace=True)
continuous_df.dropna(subset=['ret'], inplace=True)

sr = -continuous_df.ret.mean() / continuous_df.ret.std() * np.sqrt(252)
print(f'SR: {sr:.2f}')

years = continuous_df.index.values[-1] - continuous_df.index.values[0]
years = int(years) / (365 * 24 * 60 * 60) / 1e9
cagr = (continuous_df.cpnl.values[-1]) ** (1/years) - 1

print(f'CAGR: {cagr*100:.1f}% ({years:.1f}years) ({continuous_df.cpnl.values[-1]:.1f}) Wealth multiple')

px.line(continuous_df, y='cpnl', hover_data=['Close', 'Close_next', 'Close_n2'])

In [ ]:
continuous_df['ts_distance'] = continuous_df.Close - continuous_df.Close_next
print(continuous_df[['ts_distance', 'ret']].corr())
px.scatter(continuous_df, x='ts_distance', y='ret')

In [ ]:
continuous_df['ts_distance_bucket'] = (continuous_df['ts_distance']).round(0)
plot_df = continuous_df.groupby(['ts_distance_bucket']).ret.mean().to_frame('ret')
plot_df['count'] = continuous_df.groupby(['ts_distance_bucket']).size()
plot_df['std'] = continuous_df.groupby(['ts_distance_bucket']).ret.std()
plot_df['Z'] = plot_df['ret'] / plot_df['std']
px.bar(plot_df, y='Z', hover_data=['count', 'std'])


In [ ]:
continuous_df = pd.DataFrame(index=df.reset_index().date.unique()).sort_index()
continuous_df.index.name = 'date'
continuous_df = pd.merge_asof(continuous_df, left_index=True, right=roll_dates, right_on='roll_to', direction='backward')#.tail(50)
continuous_df = pd.merge(continuous_df.reset_index(), on=['Futures', 'date'], right = df.reset_index()[['date', 'Futures', 'ret', 'Close']], how='left')

continuous_df = pd.merge(continuous_df.reset_index(), left_on=['next_future', 'date'], right = df.reset_index()[['date', 'Futures', 'Close']], right_on=['Futures', 'date'], suffixes=['', '_next'], how='left')
continuous_df['term_structure'] = 'contango'
continuous_df.loc[continuous_df.Close > continuous_df.Close_next, 'term_structure'] = 'backwardation'

continuous_df['ts_distance'] = continuous_df.Close - continuous_df.Close_next
continuous_df['ts_distance_bucket'] = (continuous_df['ts_distance'] / 2).round(0)

continuous_df.loc[~continuous_df.ts_distance_bucket.isin([-2, -1, 0, 1]), 'ret'] = 0


continuous_df['cpnl'] = -continuous_df.ret # * continuous_df.ts_distance
continuous_df['cpnl'] = continuous_df['cpnl'].cumsum()
continuous_df.set_index('date', inplace=True)
continuous_df.dropna(subset=['ret'], inplace=True)

sr = -continuous_df.ret.mean() / continuous_df.ret.std() * np.sqrt(252)
print(f'SR: {sr:.2f}')

years = continuous_df.index.values[-1] - continuous_df.index.values[0]
years = int(years) / (365 * 24 * 60 * 60) / 1e9
cagr = (continuous_df.cpnl.values[-1]) ** (1/years) - 1

print(f'CAGR: {cagr*100:.1f}% ({years:.1f}years) ({continuous_df.cpnl.values[-1]:.1f}) Wealth multiple')

px.scatter(continuous_df, y='cpnl', color='ts_distance')